In [7]:
#Importing librabries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [8]:
#Reading page
r=requests.get('https://bookpage.com/')

In [9]:
#Parsing
soup=BeautifulSoup(r.text,'html.parser')

In [10]:
#Extracting values,collecting records, dataframe creation
review_links = [r'https://bookpage.com'+i.find('a')['href'] for i in soup.find_all('div', attrs={'class':'read-full'})]

rows = []

for review_link in review_links:
    r = requests.get(review_link)
    soup = BeautifulSoup(r.text,'html.parser')

    review = ''
    rev = soup.find_all('div', attrs={'class':'article-body'})
    for p in rev:
        review = review + p.find('p').text.replace('\n', ' ')

    title_author = []
    details = soup.find_all('div', attrs={'class':'book-data'})

    for t in details:
        try:
            title_author.append(t.find('h4').text.replace('\n', ''))
        except:
            pass

    author = title_author[0]
    title = title_author[1]
    genre = [i.text.replace('\n' ,'').replace(' /', ',') for i in soup.find_all('p', attrs={'class':'genre-links'})][0]

    rows.append([title, author, genre, review[1:-1]])

df = pd.DataFrame(rows, columns=['title', 'author', 'genre', 'reviews'])
df['title'] = df.title.str.replace('★ ', '')
df['author'] = df.author.str.replace('★ ', '')
df

,title,author,genre,reviews
0,Neighbors,Starcrossed,"Children's, Children's Picture Book",Tips for Teachers is a monthly column in which...
1,The All-Together Quilt,Our Little Kitchen,"Children's Picture Book, Children's",Two picture books inspired by real-life commun...
2,The Kidnapping Club,Jonathan Daniel Wells,"Nonfiction, American History, African American...","Urbane and bustling, New York City is often co..."
3,Cobble Hill,Cecily von Ziegesar,"Fiction, Popular Fiction","When a person chooses a new home, it’s rarely ..."
4,Shadows of the Short Days,Alexander Dan Vilhjálmsson,"Science Fiction & Fantasy, Fantasy","Being in a relationship is tough, even without..."
5,A Walk Around the Block,Spike Carlsen,"Nonfiction, Science & Nature",In these unprecedented days of the COVID-19 pa...


In [11]:
#saving to csv
df.to_csv('Book-reviews.csv',index=False,encoding='utf-8')